<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/library_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=1bb95d2f4c31cfe0f6bde26c8c0dc2c3640a765d6ef578afb9bee8a2ec3c7135
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 28 11:45 kaggle.json


In [3]:
!kaggle competitions download -c library-project

  0% 0.00/8.96k [00:00<?, ?B/s]
100% 8.96k/8.96k [00:00<00:00, 7.71MB/s]


In [4]:
!unzip library-project

Archive:  library-project.zip
  inflating: library_test.csv        
  inflating: library_train.csv       
  inflating: sample_submit.csv       


In [264]:
import pandas as pd
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from sklearn import preprocessing

In [265]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [266]:
learning_rate = 1e-3
training_epochs = 4000                     
batch_size = 40           
Scaler = preprocessing.MinMaxScaler()       

In [267]:
train = pd.read_csv('library_train.csv',header=None,skiprows=2)
print(train.head(10))
print(train.info())

     0   1              2   3    4   ...      8       9       10  11    12
0  시ㆍ군립  남부      수원시 선경도서관   1  567  ...  509703  427002  287365   7  1944
1  시ㆍ군립  남부      수원시서수원도서관   1  570  ...  398865  284723  142950   5  1068
2  시ㆍ군립  남부      수원시북수원도서관   1  800  ...  746202  518371  309459   7  1342
3  시ㆍ군립  남부      수원시 영통도서관   1  825  ...  776412  597632  393451   6  1211
4  시ㆍ군립  남부      수원시 중앙도서관   1  387  ...  231455  157592   81128   5  9537
5  시ㆍ군립  남부    수원시 태장마루도서관   1  582  ...  579954  441601  294095   4  1194
6  시ㆍ군립  남부  수원시 지혜샘어린이도서관   1  516  ...  363632  307166  329083   4   868
7  시ㆍ군립  남부  수원시 바른샘어린이도서관   1  500  ...  259097  238896  245654   5   868
8  시ㆍ군립  남부  수원시 슬기샘어린이도서관   1  471  ...  243058  199019  190059   6   868
9  시ㆍ군립  남부     수원시 한아름도서관   1   88  ...   76685   61405   56298   3   208

[10 rows x 13 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  -----

In [268]:
test = pd.read_csv('library_test.csv',header=None,skiprows=2)

print(test.head(10))
print(test.info())

     0   1             2   3    4   ...   7       8       9       10  11
0  시ㆍ군립  북부   고양시 아람누리도서관   1  326  ...  157  466367  426352  320636   8
1  시ㆍ군립  북부     고양시 마두도서관   1  536  ...  134  306977  173151  142997   7
2  시ㆍ군립  북부     고양시 백석도서관   1  425  ...  115  344448  233102  201259   5
3  시ㆍ군립  북부     고양시 화정도서관   1  866  ...  144  481933  313548  324145  22
4  시ㆍ군립  북부     고양시 행신도서관   1  378  ...  108  340788  231633  169154   5
5  시ㆍ군립  북부     고양시 원당도서관   1  418  ...  105  228037  174622  121731   3
6  시ㆍ군립  북부     고양시 한뫼도서관   1  391  ...  139  404896  320675  243378   5
7  시ㆍ군립  북부     고양시 대화도서관   1  435  ...  141  518542  398303  367315   8
8  시ㆍ군립  북부     고양시 풍동도서관   1  110  ...   99  207936  201749  172968   6
9  시ㆍ군립  북부  고양시 주엽어린이도서관   1   94  ...   38  244587  223274  267745   5

[10 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  -----

In [269]:
x_train = train.iloc[:,5:-1]
y_train = train.iloc[:,[-1]]
x_train = Scaler.fit_transform(x_train)
x_train = np.array(x_train)#.astype(int)
y_train = np.array(y_train)#.astype(int)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape)
print(y_train.shape)

torch.Size([180, 7])
torch.Size([180, 1])


In [270]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [271]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [272]:
linear1 = nn.Linear(7,4,bias = True)
linear2 = nn.Linear(4,4,bias = True)
linear3 = nn.Linear(4,1,bias = True)
mish = Mish()

In [273]:
nn.init.kaiming_uniform_(linear1.weight)
nn.init.kaiming_uniform_(linear2.weight)
nn.init.kaiming_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.7597, -0.2111, -0.1341,  0.6305]], requires_grad=True)

In [274]:
model = nn.Sequential(
    linear1,mish,
    linear2,mish,
    linear3
).to(device)

In [275]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [276]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    if epoch % 10 == 0:  
        print('Epoch:', '%d' % (epoch ), 'Cost =', '{:.9f}'.format(avg_cost))
print('Learning Finished')

Epoch: 0 Cost = 2746523.750000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


Epoch: 10 Cost = 2772047.250000000
Epoch: 20 Cost = 2503302.750000000
Epoch: 30 Cost = 2735251.750000000
Epoch: 40 Cost = 2534930.000000000
Epoch: 50 Cost = 2632432.750000000
Epoch: 60 Cost = 2525553.500000000
Epoch: 70 Cost = 2739196.500000000
Epoch: 80 Cost = 2569863.000000000
Epoch: 90 Cost = 2701305.000000000
Epoch: 100 Cost = 2625156.000000000
Epoch: 110 Cost = 2534329.750000000
Epoch: 120 Cost = 2715119.750000000
Epoch: 130 Cost = 2675317.500000000
Epoch: 140 Cost = 2755909.500000000
Epoch: 150 Cost = 2764623.500000000
Epoch: 160 Cost = 2569275.750000000
Epoch: 170 Cost = 2526409.250000000
Epoch: 180 Cost = 2662643.750000000
Epoch: 190 Cost = 2478993.750000000
Epoch: 200 Cost = 2603511.750000000
Epoch: 210 Cost = 2268746.500000000
Epoch: 220 Cost = 2536085.250000000
Epoch: 230 Cost = 2537063.500000000
Epoch: 240 Cost = 2365053.500000000
Epoch: 250 Cost = 2241146.500000000
Epoch: 260 Cost = 2333299.750000000
Epoch: 270 Cost = 2332015.750000000
Epoch: 280 Cost = 2216106.000000000
E

In [277]:
test = pd.read_csv('library_test.csv',header=None,skiprows=2)

print(test.head(10))
print(test.info())
test = test.iloc[:,5:]
test = Scaler.transform(test)
test = np.array(test)
test = torch.FloatTensor(test).to(device)

with torch.no_grad():
    predict = model(test)
predict

     0   1             2   3    4   ...   7       8       9       10  11
0  시ㆍ군립  북부   고양시 아람누리도서관   1  326  ...  157  466367  426352  320636   8
1  시ㆍ군립  북부     고양시 마두도서관   1  536  ...  134  306977  173151  142997   7
2  시ㆍ군립  북부     고양시 백석도서관   1  425  ...  115  344448  233102  201259   5
3  시ㆍ군립  북부     고양시 화정도서관   1  866  ...  144  481933  313548  324145  22
4  시ㆍ군립  북부     고양시 행신도서관   1  378  ...  108  340788  231633  169154   5
5  시ㆍ군립  북부     고양시 원당도서관   1  418  ...  105  228037  174622  121731   3
6  시ㆍ군립  북부     고양시 한뫼도서관   1  391  ...  139  404896  320675  243378   5
7  시ㆍ군립  북부     고양시 대화도서관   1  435  ...  141  518542  398303  367315   8
8  시ㆍ군립  북부     고양시 풍동도서관   1  110  ...   99  207936  201749  172968   6
9  시ㆍ군립  북부  고양시 주엽어린이도서관   1   94  ...   38  244587  223274  267745   5

[10 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  -----

tensor([[1745.3710],
        [1518.1801],
        [1255.2670],
        [2368.5286],
        [1225.9395],
        [ 997.6014],
        [1323.6976],
        [1495.3226],
        [1045.0718],
        [ 877.4924],
        [ 765.6346],
        [ 705.4663],
        [ 973.4793],
        [1086.7668],
        [ 997.3898],
        [ 938.4026],
        [ 931.4088],
        [1514.1166],
        [1333.9647],
        [1247.7499],
        [1356.0648],
        [1295.2972],
        [1204.4906],
        [1176.0575],
        [1158.8656],
        [1085.1202],
        [1133.8549],
        [1164.7783],
        [1022.9604],
        [2365.4526],
        [ 826.9621],
        [ 464.6377],
        [2151.8181],
        [2513.2056],
        [ 997.7238],
        [ 360.7691],
        [ 722.5806],
        [ 823.1031],
        [2378.1116],
        [ 656.4070],
        [ 534.9772],
        [ 686.9366],
        [ 726.0626],
        [ 476.4657],
        [ 293.3604],
        [ 938.4147],
        [ 893.2292],
        [ 304

In [278]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)

In [279]:
submit = pd.read_csv('sample_submit.csv')
for i in range(len(correct_prediction)):
  submit['Expected'][i]=correct_prediction[i].item()
  submit['id'][i] = i
submit.to_csv('submit.csv', mode = 'w', index = False, header = True)
submit

,id,Expected
0,0,1745
1,1,1518
2,2,1255
3,3,2368
4,4,1225
...,...,...
67,67,663
68,68,556
69,69,572
70,70,803


In [280]:
!kaggle competitions submit -c library-project -f submit.csv -m "14010974_이기택"

100% 538/538 [00:07<00:00, 71.6B/s]
Successfully submitted to SejongAI.텀프로젝트.공공도서관 예산 예측하기